In [19]:
# !pip install transformers[torch] accelerate -U
!pip install pytorch-ignite

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.2/343.2 kB 3.0 MB/s eta 0:00:00:00:01


In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

import sklearn
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity, euclidean_distances, haversine_distances, manhattan_distances
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer

import ipywidgets as widgets

# from preprocessing import *

import numpy as np
import torch

from sentence_transformers import SentenceTransformer, util

In [2]:
from datasets import load_dataset
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
)
from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers.evaluation import TripletEvaluator

from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    SentenceTransformerModelCardData,
)
from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.evaluation import TripletEvaluator
from sentence_transformers import losses

In [3]:
file_dict = {
  "train" : "../dados/augumented_train_3_negatives.csv",
  "test" : "../dados/augumented_train.csv"
}

data = load_dataset(
  'csv',
  data_files=file_dict,
  delimiter=',',
  column_names=['anchor', 'positive', 'negative'],
  skiprows=1
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [4]:
train_dataset = data["train"].select(range(11028))

eval_dataset = data["train"].select(range(11428, 17142))

In [7]:
# model = SentenceTransformer('google-bert/bert-base-multilingual-uncased')
# model = SentenceTransformer('ricardo-filho/bert-base-portuguese-cased-nli-assin-2')

model = SentenceTransformer('./models/google-bert-CachedMultipleNegativesSymmetricRanking-loss/checkpoint-1380')

In [45]:
# model_2 = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [83]:
# loss = MultipleNegativesRankingLoss(model)
# loss = losses.CachedMultipleNegativesSymmetricRankingLoss(model)
loss = losses.ContrastiveLoss(model)

In [84]:
args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir="models/google-bert-Contrastive-loss",
    # Optional training parameters:
    num_train_epochs=4,
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicates
)

Currently using DataParallel (DP) for multi-gpu training, while DistributedDataParallel (DDP) is recommended for faster training. See https://sbert.net/docs/sentence_transformer/training/distributed.html for more information.


In [44]:
trainer = SentenceTransformerTrainer(
    # output_dir="trained_models/bert-triplet",
    args=args,
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=loss,
)
trainer.train()

Step,Training Loss
500,0.032600
1000,0.015700


TrainOutput(global_step=1380, training_loss=0.019986473822939225, metrics={'train_runtime': 2119.6662, 'train_samples_per_second': 20.811, 'train_steps_per_second': 0.651, 'total_flos': 0.0, 'train_loss': 0.019986473822939225, 'epoch': 4.0})

In [38]:
# model = SentenceTransformer('google-bert/bert-base-multilingual-uncased')#('sentence-transformers/all-MiniLM-L6-v2')  # Pre-trained model
test = pd.read_parquet('../dados/preprocessed_val.parquet')
test['legal_company_name'] = test['razaosocial'] + ' - ' + test['nome_fantasia']

test.drop_duplicates(subset='legal_company_name', inplace=True)

company_names_razao = test['razaosocial'].str.lower().tolist()
company_names_fantasia = test['nome_fantasia'].str.lower().tolist()
company_names = test['legal_company_name'].str.lower().tolist()

user_inputs = test['user_input'].str.lower().tolist()

# Encode user input and dataset model.encode(names, convert_to_tensor=True)
company_embeddings_razao =  model.encode(company_names_razao, convert_to_tensor=True)
company_embeddings_fantasia = model.encode(company_names_fantasia, convert_to_tensor=True)
company_embeddings = model.encode(company_names, convert_to_tensor=True)


user_embedding = model.encode(user_inputs, convert_to_tensor=True)

# Compute cosine similarities
similarities_razao = util.pytorch_cos_sim(user_embedding, company_embeddings_razao) #[0]
similarities_fantasia = util.pytorch_cos_sim(user_embedding, company_embeddings_fantasia) #[0]
similarities = util.pytorch_cos_sim(user_embedding, company_embeddings) #[0]

ra = similarities_razao.argmax(axis=1)
fa = similarities_fantasia.argmax(axis=1)
si = similarities.argmax(axis=1)

ground_truth = test['legal_company_name'].to_list()

ra_pred = test.legal_company_name.iloc[ra.cpu()].to_list()
fa_pred = test.legal_company_name.iloc[fa.cpu()].to_list()
si_pred = test.legal_company_name.iloc[si.cpu()].to_list()

ra_ = np.array(ground_truth) == np.array(ra_pred)
fa_ = np.array(ground_truth) == np.array(fa_pred)
si_ = np.array(ground_truth) == np.array(si_pred)

ra_.mean(), fa_.mean(), si_.mean()

(0.11545711592836946, 0.31526861451460886, 0.3426013195098963)

In [47]:
import ipywidgets as widgets


def interact_cities(input_text):
    if not input_text: 
        return
        
    user_input_vectors = model.encode([input_text], convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(user_input_vectors, company_embeddings).cpu()
    values, indices = similarities.flatten().sort(descending=True)

    values = values.tolist()[:5]
    indices = indices.tolist()[:5]
    
    for va, idx in zip(values, indices):
        print(f"{test.legal_company_name.iloc[idx]:60} Score:: \x1b[6;30;42m {va:3.2f} \x1b[0m")



In [48]:
widgets.interact(interact_cities, input_text="");

interactive(children=(Text(value='', description='input_text'), Output()), _dom_classes=('widget-interact',))

In [77]:
cp = test.copy()
cp['preds'] = si_pred

In [78]:
cp = cp[cp.legal_company_name != cp.preds]

In [79]:
cp[cp.legal_company_name != cp.preds][['legal_company_name', 'user_input', 'preds']]

,legal_company_name,user_input,preds
251682,CAIXA DE ASSISTENCIA DOS ADVOGADOS DE SAO PAULO - CAASPSHOP,CAIXA ASSISTENCIA,CAIXA DE ASSISTENCIA DOS ADVOGADOS DE SAO PAULO - CAASP SANTANA
122087,BANCO BRADESCO S.A. - BRADESCO AG. PRIME VILA OLIMPIA,BRADESCO AG. P. OLINPIA,"BANCO BRADESCO S.A. - BRADESCO AG. ULIANOPOLIS, PA"
125295,"BANCO BRADESCO S.A. - BRADESCO PA SANTA MARIA DO SUACUI, MG",BRADESCO SANTA MARIA,"BANCO BRADESCO S.A. - BRADESCO PA VILA MARIA - URB. SAO PAULO, SP"
171217,MITRA ARQUIDIOCESANA DE SAO PAULO - PAROQUIA SANTO ANTONIO,MITRA SANTO,MITRA ARQUIDIOCESANA DE SAO PAULO - PAROQUIA SANTA CRUZ DAS ALMAS
120272,"BANCO BRADESCO S.A. - BRADESCO PA NUCLEO BANDEIRANTE - BRASILIA, DF",BRADESCO PA NUCLEO,"BANCO BRADESCO S.A. - BRADESCO PA PERITORO, MA"
...,...,...,...
256680,SOCIEDADE EDUCACIONAL IESP - SAO PAULO LTDA. - UNIDADE DE ENSINO IESP - COLINAS DO TOCANTIS,IESP,SOCIEDADE EDUCACIONAL IESP - SAO PAULO LTDA. - UNIDADE DE ENSINO IESP - RIBEIRAO PIRES
119076,BANCO BRADESCO S.A. - BRADESCO AG. BORBA,BREDESCO,"BANCO BRADESCO S.A. - BRADESCO PAA ROCHEDO, MS"
130583,"BANCO BRADESCO S.A. - BRADESCO PA BRAZOPOLIS, MG",BRADESCO,BANCO BRADESCO S.A. - BRADESCO
134540,"BANCO BRADESCO S.A. - BRADESCO PAA BERTOLINIA, PI",BANCO BRADESCO,BANCO BRADESCO S.A. - BRADESCO


In [80]:
new_dataset = []

for row in cp.iterrows():
    sample = row[1].copy()
    
    positive = sample[['legal_company_name', 'user_input']]
    positive['label'] = 0
    new_dataset.append(positive)
    
    negative = sample[['preds', 'user_input']]
    negative.rename(index={'preds':'legal_company_name'}, inplace=True)
    negative['label'] = 1
    new_dataset.append(negative)
    
    

In [81]:
new_augmented = pd.DataFrame(new_dataset)
new_augmented.head(20)

,legal_company_name,user_input,label
251682,CAIXA DE ASSISTENCIA DOS ADVOGADOS DE SAO PAULO - CAASPSHOP,CAIXA ASSISTENCIA,0
251682,CAIXA DE ASSISTENCIA DOS ADVOGADOS DE SAO PAULO - CAASP SANTANA,CAIXA ASSISTENCIA,1
122087,BANCO BRADESCO S.A. - BRADESCO AG. PRIME VILA OLIMPIA,BRADESCO AG. P. OLINPIA,0
122087,"BANCO BRADESCO S.A. - BRADESCO AG. ULIANOPOLIS, PA",BRADESCO AG. P. OLINPIA,1
125295,"BANCO BRADESCO S.A. - BRADESCO PA SANTA MARIA DO SUACUI, MG",BRADESCO SANTA MARIA,0
125295,"BANCO BRADESCO S.A. - BRADESCO PA VILA MARIA - URB. SAO PAULO, SP",BRADESCO SANTA MARIA,1
171217,MITRA ARQUIDIOCESANA DE SAO PAULO - PAROQUIA SANTO ANTONIO,MITRA SANTO,0
171217,MITRA ARQUIDIOCESANA DE SAO PAULO - PAROQUIA SANTA CRUZ DAS ALMAS,MITRA SANTO,1
120272,"BANCO BRADESCO S.A. - BRADESCO PA NUCLEO BANDEIRANTE - BRASILIA, DF",BRADESCO PA NUCLEO,0
120272,"BANCO BRADESCO S.A. - BRADESCO PA PERITORO, MA",BRADESCO PA NUCLEO,1


In [82]:
new_augmented.to_csv('../dados/augmented_positive_negative_labels.csv', index=False)

In [57]:
from datasets import Dataset
train_dataset = Dataset.from_dict({
    "sentence1": ["It's nice weather outside today.", "He drove to work."],
    "sentence2": ["It's so sunny.", "She walked to the store."],
    "label": [1, 0],
})

In [58]:
train_dataset

Dataset({
    features: ['sentence1', 'sentence2', 'label'],
    num_rows: 2
})

In [59]:
pd.DataFrame({
    "sentence1": ["It's nice weather outside today.", "He drove to work."],
    "sentence2": ["It's so sunny.", "She walked to the store."],
    "label": [1, 0],
})

,sentence1,sentence2,label
0,It's nice weather outside today.,It's so sunny.,1
1,He drove to work.,She walked to the store.,0


In [85]:
train_dataset = data["train"].select(range(11028))


In [86]:
train_dataset

Dataset({
    features: ['anchor', 'positive', 'negative'],
    num_rows: 11028
})

In [95]:
file_aug = {
  "train" : "../dados/augmented_positive_negative_labels.csv",
  "test" : "../dados/augmented_positive_negative_labels.csv"
}

aug = load_dataset(
  'csv',
  data_files=file_aug,
  delimiter=',',
  column_names=['legal_company_name', 'user_input', 'label'],
  skiprows=1
)

train_x = aug["train"].select(range(2000))

eval_x = aug["train"].select(range(2000, 2790))

In [96]:
trainer = SentenceTransformerTrainer(
    # output_dir="trained_models/bert-triplet",
    args=args,
    model=model,
    train_dataset=train_x,
    eval_dataset=eval_x,
    loss=loss,
)
trainer.train()

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss


TrainOutput(global_step=252, training_loss=0.007893355592848763, metrics={'train_runtime': 194.4535, 'train_samples_per_second': 41.141, 'train_steps_per_second': 1.296, 'total_flos': 0.0, 'train_loss': 0.007893355592848763, 'epoch': 4.0})

In [98]:
# model = SentenceTransformer('google-bert/bert-base-multilingual-uncased')#('sentence-transformers/all-MiniLM-L6-v2')  # Pre-trained model
test = pd.read_parquet('../dados/preprocessed_test.parquet')
test['legal_company_name'] = test['razaosocial'] + ' - ' + test['nome_fantasia']

test.drop_duplicates(subset='legal_company_name', inplace=True)

company_names_razao = test['razaosocial'].str.lower().tolist()
company_names_fantasia = test['nome_fantasia'].str.lower().tolist()
company_names = test['legal_company_name'].str.lower().tolist()

user_inputs = test['user_input'].str.lower().tolist()

# Encode user input and dataset model.encode(names, convert_to_tensor=True)
company_embeddings_razao =  model.encode(company_names_razao, convert_to_tensor=True)
company_embeddings_fantasia = model.encode(company_names_fantasia, convert_to_tensor=True)
company_embeddings = model.encode(company_names, convert_to_tensor=True)


user_embedding = model.encode(user_inputs, convert_to_tensor=True)

# Compute cosine similarities
similarities_razao = util.pytorch_cos_sim(user_embedding, company_embeddings_razao) #[0]
similarities_fantasia = util.pytorch_cos_sim(user_embedding, company_embeddings_fantasia) #[0]
similarities = util.pytorch_cos_sim(user_embedding, company_embeddings) #[0]

ra = similarities_razao.argmax(axis=1)
fa = similarities_fantasia.argmax(axis=1)
si = similarities.argmax(axis=1)

ground_truth = test['legal_company_name'].to_list()

ra_pred = test.legal_company_name.iloc[ra.cpu()].to_list()
fa_pred = test.legal_company_name.iloc[fa.cpu()].to_list()
si_pred = test.legal_company_name.iloc[si.cpu()].to_list()

ra_ = np.array(ground_truth) == np.array(ra_pred)
fa_ = np.array(ground_truth) == np.array(fa_pred)
si_ = np.array(ground_truth) == np.array(si_pred)

ra_.mean(), fa_.mean(), si_.mean()

(0.3179297597042514, 0.4579482439926063, 0.4644177449168207)

In [41]:
cp[cp.legal_company_name == cp.preds][['legal_company_name', 'user_input', 'preds']]

,legal_company_name,user_input,preds
171307,IGREJA MESSIANICA MUNDIAL DO BRASIL - IGREJA MESSIANICA MUNDIAL DO BRASIL,MESSIANICA MUNDIAL DO BRASIL,IGREJA MESSIANICA MUNDIAL DO BRASIL - IGREJA MESSIANICA MUNDIAL DO BRASIL
313589,DELICIAS DO RJ LTDA - DELICIAS DO RIO,DELISIAS RIO LTDA,DELICIAS DO RJ LTDA - DELICIAS DO RIO
124700,"BANCO BRADESCO S.A. - BRADESCO PA AVENIDA BRASILIA - URB. CURITIBA, PR",BRADESCO PA AB BRASILIA URB CURITIBA,"BANCO BRADESCO S.A. - BRADESCO PA AVENIDA BRASILIA - URB. CURITIBA, PR"
295173,ASSOCIACAO MAES DE AMOR DOWNS - AMADOS - AMADOS,AMADUS,ASSOCIACAO MAES DE AMOR DOWNS - AMADOS - AMADOS
219426,MITRA ARQUIDIOCESANA DE SAO PAULO - PAROQUIA NOSSA SENHORA DE FATIMA,ARKIDIOSESANA FATIMA,MITRA ARQUIDIOCESANA DE SAO PAULO - PAROQUIA NOSSA SENHORA DE FATIMA
...,...,...,...
289305,Y. T. NOBREGA LTDA - DISTRIBUIDORA NOBREGA,D. NOBREGA,Y. T. NOBREGA LTDA - DISTRIBUIDORA NOBREGA
128743,"BANCO BRADESCO S.A. - BRADESCO PA, ANTAS, BA",BRADESCO PA ANTAS,"BANCO BRADESCO S.A. - BRADESCO PA, ANTAS, BA"
118496,BANCO BRADESCO S.A. - AG JANDAIA DO SUL,JANDAIA DO SUL,BANCO BRADESCO S.A. - AG JANDAIA DO SUL
252166,COOPERATIVA AGRICOLA MISTA GENERAL OSORIO LTDA - LOJA COTRIBA TAPERA,COTRIBA TAPEIRA,COOPERATIVA AGRICOLA MISTA GENERAL OSORIO LTDA - LOJA COTRIBA TAPERA


## Build contrastive dataset

In [10]:
cp.shape

(2164, 20)

## Build contrastive dataset

In [15]:
# model = SentenceTransformer('google-bert/bert-base-multilingual-uncased')#('sentence-transformers/all-MiniLM-L6-v2')  # Pre-trained model
test = pd.read_parquet('../dados/preprocessed_test.parquet')
test['legal_company_name'] = test['razaosocial'] + ' - ' + test['nome_fantasia']

test.drop_duplicates(subset='legal_company_name', inplace=True)

company_names_razao = test['razaosocial'].str.lower().tolist()
company_names_fantasia = test['nome_fantasia'].str.lower().tolist()
company_names = test['legal_company_name'].str.lower().tolist()

user_inputs = test['user_input'].str.lower().tolist()

# Encode user input and dataset model.encode(names, convert_to_tensor=True)
company_embeddings_razao =  model.encode(company_names_razao, convert_to_tensor=True)
company_embeddings_fantasia = model.encode(company_names_fantasia, convert_to_tensor=True)
company_embeddings = model.encode(company_names, convert_to_tensor=True)


user_embedding = model.encode(user_inputs, convert_to_tensor=True)

# Compute cosine similarities
similarities_razao = util.pytorch_cos_sim(user_embedding, company_embeddings_razao) #[0]
similarities_fantasia = util.pytorch_cos_sim(user_embedding, company_embeddings_fantasia) #[0]
similarities = util.pytorch_cos_sim(user_embedding, company_embeddings) #[0]

ra = similarities_razao.argmax(axis=1)
fa = similarities_fantasia.argmax(axis=1)
si = similarities.argmax(axis=1)

ground_truth = test['legal_company_name'].to_list()

ra_pred = test.legal_company_name.iloc[ra.cpu()].to_list()
fa_pred = test.legal_company_name.iloc[fa.cpu()].to_list()
si_pred = test.legal_company_name.iloc[si.cpu()].to_list()

ra_ = np.array(ground_truth) == np.array(ra_pred)
fa_ = np.array(ground_truth) == np.array(fa_pred)
si_ = np.array(ground_truth) == np.array(si_pred)

ra_.mean(), fa_.mean(), si_.mean()

(0.38031423290203326, 0.5660813308687616, 0.6187615526802218)

In [17]:
top_indices = []
top_values = []

metrics = []

for row, gt_for in zip(similarities, list(range(0, similarities.shape[0]))):

    values, indices = torch.sort(row, descending=True)
    top_values.append(values[:5].cpu())
    top_indices.append(indices[:5].cpu())

    if gt_for in indices[:5].cpu():
        metrics.append(True)
    else:
        metrics.append(False)

    


In [18]:
np.array(metrics).mean()

0.7740295748613678

In [22]:
# model = SentenceTransformer('google-bert/bert-base-multilingual-uncased')#('sentence-transformers/all-MiniLM-L6-v2')  # Pre-trained model
train = pd.read_parquet('../dados/preprocessed_train.parquet')
train['legal_company_name'] = train['razaosocial'] + ' - ' + train['nome_fantasia']

train.drop_duplicates(subset='legal_company_name', inplace=True)

company_names_razao = train['razaosocial'].str.lower().tolist()
company_names_fantasia = train['nome_fantasia'].str.lower().tolist()
company_names = train['legal_company_name'].str.lower().tolist()

user_inputs = train['user_input'].str.lower().tolist()

# Encode user input and dataset model.encode(names, convert_to_tensor=True)
company_embeddings_razao =  model.encode(company_names_razao, convert_to_tensor=True)
company_embeddings_fantasia = model.encode(company_names_fantasia, convert_to_tensor=True)
company_embeddings = model.encode(company_names, convert_to_tensor=True)


user_embedding = model.encode(user_inputs, convert_to_tensor=True)

# Compute cosine similarities
similarities_razao = util.pytorch_cos_sim(user_embedding, company_embeddings_razao) #[0]
similarities_fantasia = util.pytorch_cos_sim(user_embedding, company_embeddings_fantasia) #[0]
similarities = util.pytorch_cos_sim(user_embedding, company_embeddings) #[0]

In [23]:
ra = similarities_razao.argmax(axis=1)
fa = similarities_fantasia.argmax(axis=1)
si = similarities.argmax(axis=1)

In [24]:
ground_truth = train['legal_company_name'].to_list()

ra_pred = train.legal_company_name.iloc[ra.cpu()].to_list()
fa_pred = train.legal_company_name.iloc[fa.cpu()].to_list()
si_pred = train.legal_company_name.iloc[si.cpu()].to_list()

In [25]:
ra_ = np.array(ground_truth) == np.array(ra_pred)
fa_ = np.array(ground_truth) == np.array(fa_pred)
si_ = np.array(ground_truth) == np.array(si_pred)

ra_.mean(), fa_.mean(), si_.mean()

(0.30574028701435074, 0.5929296464823242, 0.6909345467273363)

In [15]:
ra_ = np.array(ground_truth) == np.array(ra_pred)
fa_ = np.array(ground_truth) == np.array(fa_pred)
si_ = np.array(ground_truth) == np.array(si_pred)

ra_.mean(), fa_.mean(), si_.mean()

(0.3029401470073504, 0.5719285964298215, 0.6727336366818341)

In [35]:
# model = SentenceTransformer('google-bert/bert-base-multilingual-uncased')#('sentence-transformers/all-MiniLM-L6-v2')  # Pre-trained model
test = pd.read_parquet('../dados/preprocessed_val.parquet')
test['legal_company_name'] = test['razaosocial'] + ' - ' + test['nome_fantasia']

test.drop_duplicates(subset='legal_company_name', inplace=True)

company_names_razao = test['razaosocial'].str.lower().tolist()
company_names_fantasia = test['nome_fantasia'].str.lower().tolist()
company_names = test['legal_company_name'].str.lower().tolist()

user_inputs = test['user_input'].str.lower().tolist()

# Encode user input and dataset model.encode(names, convert_to_tensor=True)
company_embeddings_razao =  model.encode(company_names_razao, convert_to_tensor=True)
company_embeddings_fantasia = model.encode(company_names_fantasia, convert_to_tensor=True)
company_embeddings = model.encode(company_names, convert_to_tensor=True)


user_embedding = model.encode(user_inputs, convert_to_tensor=True)

# Compute cosine similarities
similarities_razao = util.pytorch_cos_sim(user_embedding, company_embeddings_razao) #[0]
similarities_fantasia = util.pytorch_cos_sim(user_embedding, company_embeddings_fantasia) #[0]
similarities = util.pytorch_cos_sim(user_embedding, company_embeddings) #[0]

ra = similarities_razao.argmax(axis=1)
fa = similarities_fantasia.argmax(axis=1)
si = similarities.argmax(axis=1)

ground_truth = test['legal_company_name'].to_list()

ra_pred = test.legal_company_name.iloc[ra.cpu()].to_list()
fa_pred = test.legal_company_name.iloc[fa.cpu()].to_list()
si_pred = test.legal_company_name.iloc[si.cpu()].to_list()

ra_ = np.array(ground_truth) == np.array(ra_pred)
fa_ = np.array(ground_truth) == np.array(fa_pred)
si_ = np.array(ground_truth) == np.array(si_pred)

ra_.mean(), fa_.mean(), si_.mean()

(0.11734213006597549, 0.324693685202639, 0.34872761545711595)

In [36]:
ra_pred

['CAIXA DE ASSISTENCIA DOS ADVOGADOS DE SAO PAULO - CAASPSHOP',
 'IGREJA MESSIANICA MUNDIAL DO BRASIL - IGREJA MESSIANICA MUNDIAL DO BRASIL',
 'BANCO BRADESCO S.A. - BRADESCO AG. PRIME VILA OLIMPIA',
 'BANCO BRADESCO S.A. - BRADESCO AG. PRIME VILA OLIMPIA',
 'DELICIAS DO RJ LTDA - DELICIAS DO RIO',
 'BANCO BRADESCO S.A. - BRADESCO AG. PRIME VILA OLIMPIA',
 'MITRA ARQUIDIOCESANA DE SAO PAULO - PAROQUIA SANTO ANTONIO',
 'ASSOCIACAO MAES DE AMOR DOWNS - AMADOS - AMADOS',
 'BANCO BRADESCO S.A. - BRADESCO AG. PRIME VILA OLIMPIA',
 'BANCO BRADESCO S.A. - BRADESCO AG. PRIME VILA OLIMPIA',
 'BANCO BRADESCO S.A. - BRADESCO AG. PRIME VILA OLIMPIA',
 'PILLAR ESTRUTURAS PRE-FABRICADAS LTDA - PILLAR ESTRUTURAS',
 'MITRA ANGELOPOLITANA - FRATERNO AUXILIO DO CLERO',
 'BANCO BRADESCO S.A. - BRADESCO AG. PRIME VILA OLIMPIA',
 'BANCO BRADESCO S.A. - BRADESCO AG. PRIME VILA OLIMPIA',
 'BANCO BRADESCO S.A. - BRADESCO AG. PRIME VILA OLIMPIA',
 'C.VALE - COOPERATIVA AGROINDUSTRIAL - C.VALE',
 'BANCO BRADESC

In [37]:
ground_truth

['CAIXA DE ASSISTENCIA DOS ADVOGADOS DE SAO PAULO - CAASPSHOP',
 'IGREJA MESSIANICA MUNDIAL DO BRASIL - IGREJA MESSIANICA MUNDIAL DO BRASIL',
 'BANCO BRADESCO S.A. - BRADESCO AG. PRIME VILA OLIMPIA',
 'BANCO BRADESCO S.A. - BRADESCO PA SANTA MARIA DO SUACUI, MG',
 'DELICIAS DO RJ LTDA - DELICIAS DO RIO',
 'BANCO BRADESCO S.A. - BRADESCO PA AVENIDA BRASILIA - URB. CURITIBA, PR',
 'MITRA ARQUIDIOCESANA DE SAO PAULO - PAROQUIA SANTO ANTONIO',
 'ASSOCIACAO MAES DE AMOR DOWNS - AMADOS - AMADOS',
 'BANCO BRADESCO S.A. - BRADESCO PA NUCLEO BANDEIRANTE - BRASILIA, DF',
 'BANCO BRADESCO S.A. - BRADESCO AG. PARINTINS EST UNIF',
 'BANCO BRADESCO S.A. - BRADESCO PA AVENIDA ADOLFO PINHEIRO - URB SAO PAULO, SP',
 'DIMED S/A - DISTRIBUIDORA DE MEDICAMENTOS - PANVEL FARMACIAS',
 'MITRA ARQUIDIOCESANA DE SAO PAULO - PAROQUIA NOSSA SENHORA DE FATIMA',
 'BANCO BRADESCO S.A. - BRADESCO NEXT XVIII',
 'BANCO BRADESCO S.A. - AG PALMEIRA DOS INDIOS',
 'BANCO BRADESCO S.A. - BRADESCO PAA CRISTOPOLIS, BA',
 'VE